In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_x = train.drop(['label'], axis=1)
train_y = train['label']

In [5]:
train_x = train_x.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

In [6]:
train_x = train_x/255.
test = test/255.

In [7]:
train_x.shape, test.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.1, random_state=42)

In [9]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax'),
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                   

In [11]:
optimizerr = tf.keras.optimizers.Adam()
losss = tf.keras.losses.SparseCategoricalCrossentropy()

In [12]:
model.compile(optimizer=optimizerr, loss=losss, metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train, epochs=8)

Epoch 1/8
1182/1182 [==============================] - 34s 29ms/step - loss: 0.0235 - accuracy: 0.9937
Epoch 2/8
1182/1182 [==============================] - 34s 28ms/step - loss: 0.0283 - accuracy: 0.9930
Epoch 3/8
1182/1182 [==============================] - 34s 29ms/step - loss: 0.0238 - accuracy: 0.9936
Epoch 4/8
1182/1182 [==============================] - 34s 29ms/step - loss: 0.0193 - accuracy: 0.9951
Epoch 5/8
1182/1182 [==============================] - 34s 29ms/step - loss: 0.0268 - accuracy: 0.9931
Epoch 6/8
1182/1182 [==============================] - 35s 29ms/step - loss: 0.0230 - accuracy: 0.9936
Epoch 7/8
1182/1182 [==============================] - 34s 29ms/step - loss: 0.0182 - accuracy: 0.9951
Epoch 8/8
1182/1182 [==============================] - 34s 29ms/step - loss: 0.0244 - accuracy: 0.9932


In [18]:
y_pred = model.predict(X_test)
y_pred =np.argmax(y_pred,axis=1)

132/132 [==============================] - 1s 9ms/step


In [19]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred, y_test))

0.9909523809523809


In [20]:
results=model.predict(test)
results=np.argmax(results,axis=1)

results=pd.Series(results,name="Label")

875/875 [==============================] - 8s 9ms/step


In [21]:
submission = pd.DataFrame({"ImageId":[i+1 for i in range(len(results))],"Label":results})
submission.to_csv('submission.csv', index=False)